In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2
from collections import Counter
import random
from utils import create_dataset
from utils import load_model, save_checkpoint
from utils import AveragePrecision
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
from torch.autograd import Variable

## PreProcess

In [8]:
rating_data = pd.read_csv("data/rating_train.csv")
rating_data = rating_data.drop(['date'], axis=1).drop_duplicates()
rating_data['label'] = 1
user_ids = rating_data.userid.drop_duplicates()

In [10]:
user_food_score = [{} for i in range(rating_data.userid.max()+1)]
for u, f in zip(rating_data['userid'], rating_data['foodid']):
    if user_food_score[u].get(f) != None:
        user_food_score[u][f] += 1
    else:
        user_food_score[u][f] = 1
    
for i in user_ids:
    all_val = list(user_food_score[i].values())
    for k, v in user_food_score[i].items():
        user_food_score[i][k] = v / max(all_val)

In [14]:
valid_percentage = 0.5
user_id_train, user_id_valid, food_id_train, food_id_valid, Y_train, Y_valid = create_dataset(user_food_score, rating_data, valid_percentage)

In [15]:
train_loader = DataLoader(
        dataset=TensorDataset(
            torch.from_numpy(user_id_train),
            torch.from_numpy(food_id_train),
            torch.from_numpy(Y_train)),
        batch_size=1024,
        shuffle=True,
        num_workers=8)

In [16]:
not_eaten = {}
for u in user_ids:
    t_start_idx = np.where(user_id_train == u)[0][0]
    t_end_idx = np.where(user_id_train == u)[0][-1]
    v_start_idx = np.where(user_id_valid == u)[0][0]
    v_end_idx = np.where(user_id_valid == u)[0][-1]
    #tmp = []
    #for i in range(v_start_idx, v_end_idx+1):
        #if (food_id_valid[i] not in food_id_train[t_start_idx: t_end_idx+1]) and (food_id_valid[i] not in tmp):
        #    tmp.append(food_id_valid[i])
    #print(u)
    #print(t_end_idx)
    not_eaten[u] = np.setdiff1d(food_id_valid[v_start_idx: v_end_idx+1], food_id_train[t_start_idx: t_end_idx+1])

## Model

In [17]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_foods, n_dim):
        super(MatrixFactorization, self).__init__()
        self.user_embed_layer = nn.Embedding(n_users, n_dim).cuda()
        self.food_embed_layer = nn.Embedding(n_foods, n_dim).cuda()
        self.user_bias_embed_layer = nn.Embedding(n_users, 1).cuda()
        self.food_bias_embed_layer = nn.Embedding(n_foods, 1).cuda()
        
    def forward(self, user_train, food_train):
        user_vec = self.user_embed_layer(user_train.cuda())
        user_vec = nn.Dropout(0.3)(user_vec)
        user_vec = nn.Sigmoid()(user_vec)
        food_vec = self.food_embed_layer(food_train.cuda())
        food_vec = nn.Dropout(0.3)(food_vec)
        food_vec = nn.Sigmoid()(food_vec)
        user_bias = self.user_bias_embed_layer(user_train.cuda()).view(-1)
        food_bias = self.food_bias_embed_layer(food_train.cuda()).view(-1)
        
        lamb = 0.01
        reg1 = lamb * torch.norm(torch.cat([x.view(-1) for x in self.parameters()]), 1)
        reg2 = lamb * torch.norm(torch.cat([x.view(-1) for x in self.parameters()]), 2)
        
        return ((user_vec * food_vec).sum(1) + user_bias + food_bias).cuda(), reg1+reg2

In [20]:
n_dim = 10
model = MatrixFactorization(max(user_ids)+1, rating_data.foodid.max()+1, n_dim).cuda()
loss_func = nn.MSELoss()
opt_u = torch.optim.Adam(list(model.user_embed_layer.parameters()), lr=0.01)
opt_f = torch.optim.Adam(list(model.food_embed_layer.parameters()), lr=0.01)
opt_b = torch.optim.Adam(list(model.food_bias_embed_layer.parameters())+list(model.user_bias_embed_layer.parameters()), lr=0.001)
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.002)

## Training

In [23]:
epochs = 30
print_every = 1
infer_every = 20
best = 1e10
best_epoch = 1
best_map = 0
for e in range(1, epochs+1):
    t_loss = 0
    for i ,(user, food, rate) in enumerate(train_loader):
        user, food, rate = user.cuda(), food.cuda(), rate.cuda().float()
        output, regu_loss = model(user, food)
        loss = loss_func(output, rate)
        t_loss += loss
        
    #validation
    v_out, _ = model(user_id_valid.cuda(), food_id_valid.cuda())
    v_loss = loss_func(v_out, Y_valid)
    
    opt_u.zero_grad()
    opt_f.zero_grad()
    opt_b.zero_grad()
    loss.backward()
    opt_u.step()
    opt_f.step()
    opt_b.step()
    
    all_ap, _ = infer()
    MAP = np.mean(all_ap)
    if e % print_every == 0:
        print("Epoch {}, train_loss = {}, valid_loss = {}, map = {}, best_map = {}".format(e, t_loss, v_loss, MAP))

RuntimeError: CUDA error: out of memory

In [9]:
model,_,_,_ = load_model('model.h5.best', model)

=> loading checkpoint model.h5.best
=> loaded checkpoint model.h5.best (epoch: 252)


## Inference

In [9]:
def infer(): 
    F = np.arange(len(food_data), dtype=int)
    F = [i for i in range(len(food_data))]
    all_ans = []
    all_score = []
    for u in user_ids:
        ans = []
        U = [u for i in range(len(food_data))]
        res = model(Variable(torch.LongTensor(U)), Variable(torch.LongTensor(F))).cpu().data.numpy()
        res = res.flatten()
        D = {k: v for (k, v) in zip(F, res)}
        sorted_D = sorted(D.items(), key=lambda d: d[1], reverse=True)
        for x in sorted_D:
            if len(ans) >= 20: break
            if x[0] in user_food_score[u].keys():
                continue
            ans.append(x[0])
        all_score.append(AveragePrecision(not_eaten[u], ans, len(ans)))
        all_ans.append(ans) 
    return all_score, all_ans

In [16]:
not_eaten[6]

array([122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
       148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160,
       161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173,
       174, 175])

## Generate Output

In [63]:
with open("pred.csv", 'w') as f:
    f.write("userid,foodid\n")
    for i in range(len(user_ids)):
        f.write("{},".format(user_ids[i]))
        for j in all_ans[i]:
            f.write("{} ".format(j))
        f.write('\n')